# Assignment 2

#### First Name: Flàvia 
#### Last Name: Ferrús Marimón


## 1. Load Data from JSON

In [1]:

from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Twitter Analysis")\
.getOrCreate()


In [5]:
df_twitter = spark.read.json("corona_tweet_new.json")

In [6]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [7]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)


In [8]:
from pyspark.sql.functions import col

# Select only the required columns from the nested user struct column
user_cols = ["id_str", "followers_count", "friends_count", "created_at"]
user_sub_df = df_twitter.select([col("user."+c).alias("user_"+c) for c in user_cols])

# Join the original DataFrame with the extracted user sub-DataFrame
df_twitter1 = df_twitter.join(user_sub_df, how="left")

# Drop the original nested user column
df_twitter1 = df_twitter1.drop("user")

# Print the resulting schema
df_twitter1.printSchema()


root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [9]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter1.count()

58905625

In [11]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regular expresion and the group would be 1 for this regular expression.
#(4 points)
#from pyspark.sql.functions import regexp_extract, col

#df_twitter=df_twitter.<FILL_IN>
#df_twitter.select(col('extracted_source'),col('source')).show()

In [12]:
from pyspark.sql.functions import regexp_extract, col

# Extract the source label from the source column using a regular expression
df_twitter2 = df_twitter1.withColumn("extracted_source", regexp_extract(col("source"), '<a [^>]+>([^<]+)', 1))

# Select the extracted_source and source columns and show the resulting DataFrame
df_twitter2.select(col('extracted_source'), col('source')).show()


+----------------+--------------------+
|extracted_source|              source|
+----------------+--------------------+
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
| Twitter Web App|<a href="https://...|
+----------------+--------------------+
only showing top 20 rows



In [13]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter2.rdd.map(tuple)

In [14]:
# Create a temporay table in memory with name as twitter (1 point)
df_twitter2.createOrReplaceTempView("twitter")

In [15]:
cols = df_twitter2.columns
print(cols[10])

user_id_str


In [16]:
rdd_twitter.take(2)

[('Mon Apr 20 15:01:57 +0000 2020',
  5545,
  [],
  '1252251164227362821',
  None,
  None,
  'India',
  3460,
  5477,
  '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  '93612837',
  121,
  759,
  'Mon Nov 30 11:38:08 +0000 2009',
  'Twitter Web App'),
 ('Mon Apr 20 15:01:57 +0000 2020',
  5545,
  [],
  '1252251164227362821',
  None,
  None,
  'India',
  3460,
  5477,
  '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  '346443880',
  208,
  1196,
  'Mon Aug 01 08:15:42 +0000 2011',
  'Twitter Web App')]

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

##### Using RDD

In [12]:
# Using RDD

## We need to get the total number of rows with unique user_id_str

import pyspark
#sc = pyspark.SparkContext('local[*]')
# Display the type of the Spark Context sc
#type(sc)
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Big Data Assisgnment 2") \
    .getOrCreate()
sc=spark.sparkContext

In [19]:
## Find the index of the column where we have the user id
rdd_twitter.first()[10]

## This is the id 

'93612837'

In [20]:
#rdd_twitter.user_id_str.take(1)

In [21]:
#unique_users = rdd_twitter.map(lambda x: x[10]).distinct().count()
#print(unique_users)

Recall that this code takes a lot of memory, since the function `distinct` is not the optimum way to compute the unique number of users from the dataset. 

Observe that we can get the unique number of users by applying an alternative `rdd` code in spark

In [22]:
users = rdd_twitter.map(lambda x: (x[10], 1))
users.take(2)


[('93612837', 1), ('346443880', 1)]

In [23]:
unique_users2 = users.reduceByKey(lambda a, b : a+b)
unique_users2.take(2)

[('93612837', 15894), ('2781749028', 15894)]

In [24]:
uniqueUserCount = unique_users2.count()
print ('Unique users: %d' % uniqueUserCount)

Unique users: 14094


`Unique users: 14094`

##### Using DataFrame

In [ ]:
# Using DataFrame
#unique_users_df = df_twitter2.select("user_id_str").distinct().count()
#print("Total number of unique users: ", unique_users_df)

Again this code may take longer, since it considers the `distinct()` function. We can follow an approach similar to the previous one with the `rdd` object by considering: 

In [26]:
from pyspark.sql.functions import count

users_df = df_twitter2.groupBy("user_id_str")
unique_users2_df = users_df.agg(count("*").alias("count"))
uniqueUserCount_df = unique_users2_df.count()
print('Unique users: %d' % uniqueUserCount_df)


Unique users: 14094


##### Using `spark.sql` and the temporary table

In [28]:
# Using spark.sql and the temporay table.

## We use the table we have created by running the command:
#df_twitter2.createOrReplaceTempView("twitter")

# Count the total number of unique users
uniqueUserCount_sql = spark.sql("SELECT COUNT(DISTINCT user_id_str) FROM twitter")

# Print the result
print(uniqueUserCount_sql.first()[0])


14094


#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

##### Using RDD

Since we already have the number of tweets per user, we do not need to compute it again, we just need to count the number of users that actually have more than 1 tweet:

In [25]:
unique_users2.take(2)
users_more1 = unique_users2.filter(lambda x: x[1] > 1)
more1_UserCount = users_more1.count()
print ('Unique hosts: %d' % more1_UserCount)

Unique hosts: 14094


##### Using DataFrame

Similarly to the previous case, we do not need to compute the aggregation again:

In [27]:
from pyspark.sql.functions import count

# Count the number of unique users with more than one tweet
#uniqueUserCount_df = df_twitter2.groupBy("user_id").agg(count("*").alias("count")).filter("count > 1").count()
users_more1_df = unique_users2_df.filter("count > 1")
more1_UserCount_df = users_more1_df.count()
print('Unique hosts with more than one tweet: %d' % more1_UserCount_df)


Unique hosts with more than one tweet: 14094


##### Using spark.sql and the temporay table.

In [30]:
# Count the number of unique users with more than one tweet using SQL
more1_UserCount_sql = spark.sql("SELECT COUNT(DISTINCT user_id_str) FROM twitter HAVING COUNT(*) > 1")
#more1_UserCount_sql = spark.sql("SELECT COUNT(DISTINCT user_id_str) FROM twitter GROUP BY user_id_str HAVING COUNT(*) > 1")

# Print the result
print('Unique hosts with more than one tweet: %d' % more1_UserCount_sql.first()[0])

Unique hosts with more than one tweet: 14094


Observe that here we have used the `HAVING` command from the `spark.sql` set of functions, instead of the `WHERE`. This is because the `WHERE` clause is used to filter rows before they are grouped by an aggregate function. It operates on individual rows, and its condition is evaluated before the grouping is performed. The `WHERE` clause can only reference columns in the table being queried, and cannot reference the results of aggregate functions.

Meanwhile, the `HAVING` clause is used to filter groups after they are created by an aggregate function. It operates on groups of rows that have been created by a `GROUP BY` clause, like in this case, and its condition is evaluated after the grouping is performed. The `HAVING` clause can reference aggregate functions, such as `COUNT()`, and can be used to filter groups based on the results of those functions.

#### 2.3 Get total number unique extracted_source (1 point each)

Now we have to repeat the previous approach but taking into account the `extracted_source` columns this time, isntead of the `user_id_str`. In order to do so, notice that this column corresponds to the last column of the data set created `twitter_df2`. Therefore, we can modify the code and consider the alternative column:

In [31]:
cols = df_twitter2.columns
print(cols)
print(cols[10])
print(len(cols))
print(cols[14])

['created_at', 'favorite_count', 'hashtags', 'id', 'in_reply_to_status_id', 'in_reply_to_user_id_str', 'location', 'reply_count', 'retweet_count', 'source', 'user_id_str', 'user_followers_count', 'user_friends_count', 'user_created_at', 'extracted_source']
user_id_str
15
extracted_source


##### Using RDD

In [32]:
sources = rdd_twitter.map(lambda x: (x[14], 1))
unique_sources = sources.reduceByKey(lambda a, b : a+b)
unique_sourcesCount = unique_sources.count()
print ('Unique sources: %d' % unique_sourcesCount)

Unique sources: 133


##### Using DataFrame

In [34]:
sources_df = df_twitter2.groupBy("extracted_source")
unique_sources_df = sources_df.agg(count("*").alias("count"))
unique_sourcesCount_df = unique_sources_df.count()
print('Unique sources: %d' % unique_sourcesCount_df)

Unique sources: 133


##### Using spark.sql and the temporay table.

In [40]:
unique_sourcesCount_sql = spark.sql("SELECT COUNT(DISTINCT extracted_source) FROM twitter")
print('Unique sources: %d' % unique_sourcesCount_sql.first()[0])

Unique sources: 133


#### 2.4 Get top 5 most used extracted_source

Here we can order the data once we have already aggregated the number of times each `extracted_source` appears, and then take the top 5 from the sorted list:

##### Using RDD (5 points)

In [33]:
top_sources = unique_sources.takeOrdered(5, lambda s: -1 * s[1])
print('Top 5 sources: %s' % top_sources)

Top 5 sources: [('Twitter for Android', 99528228), ('Twitter for iPhone', 90564012), ('Twitter Web App', 45742932), ('Twitter for iPad', 6802632), ('Twitter Web Client', 2161584)]


##### Using DataFrame (2 points)

In [35]:
top_sources_df = unique_sources_df.orderBy("count", ascending=False).limit(5)
print('Top 5 sources: %s' % top_sources_df)

Top 5 sources: DataFrame[extracted_source: string, count: bigint]


In [43]:
top_sources_list_df = top_sources_df.collect()

print("Top 5 sources:")
for row in top_sources_list_df:
    print(row[0])

Top 5 sources:
Twitter for Android
Twitter for iPhone
Twitter Web App
Twitter for iPad
Twitter Web Client


##### Using spark.sql and the temporay table. (2 points)

In [41]:
# Query to get the top 5 most used sources
top_sources_sql = spark.sql("SELECT extracted_source, COUNT(*) AS count "
                            "FROM twitter "
                            "GROUP BY extracted_source "
                            "ORDER BY count DESC "
                            "LIMIT 5")

print('Top 5 sources: %s' % top_sources_sql)

Top 5 sources: DataFrame[extracted_source: string, count: bigint]


In [42]:
top_sources_list = top_sources_sql.collect()

print("Top 5 sources:")
for row in top_sources_list:
    print(row[0])


Top 5 sources:
Twitter for Android
Twitter for iPhone
Twitter Web App
Twitter for iPad
Twitter Web Client


#### 2.5 Get count of distinct hastags used ( 5 point each) 

We may consider now the column corresponding to the `hastags`:

In [44]:
print(cols[2])

hashtags


Observe however that this could be more complex, since the `hastags` column contains all the possible hastags that are used in the particular tweet. Therefore, we may need to consider the different elements of the list as the particular hastags we want to count. 

Therefore, if we try to compute a similar code to the previous ones we will get the corresponding error since lists are not hashable and the `reduceByKey` function cannot operate if the key is not hashable. Thus, in order to solve this problem the `flatMap` function is used, taking the different possible hastags within each list from the column as the key of our `unique_hastags` rdd object. This is:

##### Using RDD

In [46]:
#hastags = rdd_twitter.map(lambda x: (x[2], 1))
hashtags = rdd_twitter.flatMap(lambda x: [(tag, 1) for tag in x[2]])
hashtags.take(10)

[('boxing', 1),
 ('kickboxing', 1),
 ('covid19', 1),
 ('corona', 1),
 ('SelfQuarantine', 1),
 ('boxing', 1),
 ('kickboxing', 1),
 ('covid19', 1),
 ('corona', 1),
 ('SelfQuarantine', 1)]

In [47]:
unique_hashtags = hashtags.reduceByKey(lambda a, b : a+b)
unique_hashtagsCount = unique_hashtags.count()
print ('Unique hastags: %d' % unique_hashtagsCount)

Unique hastags: 1215


##### Using DataFrame

I follow a similar approach for the case of the dataframe, here the `explode()` function is used to flatten the list of hashtags into separate rows.

In [49]:
from pyspark.sql.functions import explode

hashtags_df_1 = df_twitter2.select(explode("hashtags").alias("hashtag"))
hashtags_df = hashtags_df_1.groupBy("hashtag")
unique_hashtags_df = hashtags_df.agg(count("*").alias("count"))
unique_hashtagsCount_df = unique_hashtags_df.count()
print('Unique hashtags: %d' % unique_hashtagsCount_df)


Unique hashtags: 1215


##### Using spark.sql and the temporay table.

Similarly, for the sql code, we use the `LATERAL VIEW` clause to explode the hashtags array column and create a new row for each hashtag, and then we count the distinct hashtags using the `COUNT(DISTINCT)` function like in the previous cases.

In [52]:
unique_hashtagsCount_sql = spark.sql("SELECT COUNT(DISTINCT hashtag) FROM twitter LATERAL VIEW EXPLODE(hashtags) AS hashtag")
print('Unique hashtags: %d' % unique_hashtagsCount_sql.first()[0])


Unique hashtags: 1215


#### 2.6 Get top 5 hashtags

Once that we already have the corresponding objects with the count for every `hashtag` it is direct to get the top 5, by applying similar code to the one on section 2.4

##### Using RDD (4 points)

In [48]:
top_hashtags = unique_hashtags.takeOrdered(5, lambda s: -1 * s[1])
print('Top 5 hashtags: %s' % top_hashtags)

Top 5 hashtags: [('طبق_القدرات_للثانويه_ياريس', 6119190), ('Corona', 5070186), ('OilPrice', 3989394), ('COVID19', 1986750), ('corona', 1954962)]


Somethign happened with this count, arabic numbers considered?

##### Using DataFrame (2 points)

In [50]:
top_hashtags_df = unique_hashtags_df.orderBy("count", ascending=False).limit(5)
print('Top 5 hashtags: %s' % top_hashtags_df)

Top 5 hashtags: DataFrame[hashtag: string, count: bigint]


In [51]:
top_hashtags_list_df = top_hashtags_df.collect()

print("Top 5 hashtags:")
for row in top_hashtags_list_df:
    print(row[0])

Top 5 hashtags:
طبق_القدرات_للثانويه_ياريس
Corona
OilPrice
COVID19
corona


The first most used hashtag appears to be in arabic, but the count is correct.

##### Using spark.sql and the temporay table. (2 points)

In [63]:
unique_hashtagsCount_sql = spark.sql("SELECT DISTINCT hashtag "
                                     "FROM twitter LATERAL VIEW EXPLODE(hashtags) AS hashtag")


In [64]:
_hashtagsCount_sql = spark.sql("SELECT hashtag "
                                     "FROM twitter LATERAL VIEW EXPLODE(hashtags) AS hashtag")


In [65]:
_hashtagsCount_sql.show()

+--------------+
|       hashtag|
+--------------+
|        boxing|
|    kickboxing|
|       covid19|
|        corona|
|SelfQuarantine|
|        boxing|
|    kickboxing|
|       covid19|
|        corona|
|SelfQuarantine|
|        boxing|
|    kickboxing|
|       covid19|
|        corona|
|SelfQuarantine|
|        boxing|
|    kickboxing|
|       covid19|
|        corona|
|SelfQuarantine|
+--------------+
only showing top 20 rows



In [66]:
_hashtagsCount_sql.createOrReplaceTempView("hashtags")
top_hashtags_sql = spark.sql("SELECT hashtag as hashtag, COUNT(*) as count "
                             "FROM hashtags "
                             "GROUP BY hashtag "
                             "ORDER BY count DESC "
                             "LIMIT 5 ")
print('Top 5 hashtags: ')
top_hashtags_sql.show()


Top 5 hashtags: 
+--------------------+-------+
|             hashtag|  count|
+--------------------+-------+
|طبق_القدرات_للثان...|6119190|
|              Corona|5070186|
|            OilPrice|3989394|
|             COVID19|1986750|
|              corona|1954962|
+--------------------+-------+



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [15]:
print(cols[8])
print(cols[3])

retweet_count
id


Notice that here we need to get the different tweets we have, because if we directly filter by the tweets that have `retweet_count` greater than 100 we may count the tweets several times, when it is actually the same. Therefore, we may need to group by the tweet id, and then filter by the retweet count. 

However, we may see whether the retweet count increasses when the tweet is retweeted, or whether it starts on 0 again. If it starts on 0 and the tweet is apparently counted as a new tweet we need to add the retweet count. If not, we just need to group by tweet Id, since they all should have the same retweet count. 

##### Using RDD

In order to check which case we are in front of, what we should do is consider:

In [68]:
tweets = rdd_twitter.map(lambda x: (x[3], (x[8], 1)))
tweets.take(10)
#unique_tweets = tweets.reduceByKey(lambda a, b : a[1]+b[1])
#unique_tweets.take(10)

[('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1)),
 ('1252251164227362821', (5477, 1))]

They all have the same retweet count when considering the tweet ID, even when different users retweeted it. 

If they all have the same count when the tweet id is the same:

In [73]:
thresh = 100
tweets = rdd_twitter.map(lambda x: (x[3], x[8]))
tweets.take(2)
#unique_tweets = tweets.groupByKey()
unique_tweets = tweets.reduceByKey(lambda a, b : max(a,b))
unique_tweets.take(2)


[('1252251164227362821', 5477), ('1252251166504824833', 6374)]

In [74]:
retweeted = unique_tweets.filter(lambda x : x[1] > thresh)
retweeted.take(2)

[('1252251164227362821', 5477), ('1252251166504824833', 6374)]

In [75]:
number_retweeted = retweeted.count()
print('Number of tweets retweeted more than 100 times: %d' % number_retweeted)

Number of tweets retweeted more than 100 times: 15706


If somehow the retweet count does not take into account all the aggregated retweets, then we have to aggregate them: (not needed)

In [ ]:
#thresh = 100
#tweets = rdd_twitter.map(lambda x: (x[3], x[8]))
#unique_tweets = tweets.reduceByKey(lambda a, b : a+b)

#retweeted = unique_tweets.filter(lambda x : x[1] > thresh)
#number_retweeted = retweeted.count()
#print('Number of tweets retweeted more than 100 times: %d' % number_retweeted)

##### Using DataFrame


In [17]:
from pyspark.sql.functions import count
from pyspark.sql.functions import max


tweets_df = df_twitter2.groupBy("id")
unique_tweets_df = tweets_df.agg(max("retweet_count").alias("count"))


In [18]:
retweeted_df = unique_tweets_df.filter("count > 100")
number_retweeted_df = retweeted_df.count()
print('Number of tweets retweeted more than 100 times: %d' % number_retweeted_df)

Number of tweets retweeted more than 100 times: 15706


##### Using spark.sql and the temporay table.


In [19]:
# Count the number of unique users with more than one tweet using SQL
number_retweeted_sql = spark.sql("SELECT COUNT(DISTINCT id) FROM twitter HAVING MAX(retweet_count) > 100")

# Print the result
print('Number of tweets retweeted more than 100 times: %d' % number_retweeted_sql.first()[0])

Unique hosts with more than one tweet: 15847


This number does not correspond to the previous results, therefore a different approach is conducted so the aggregation of the tweets per `id` is implemented correctly:

In [21]:
count_retweeted_sql = spark.sql("SELECT COUNT(*) FROM (SELECT id, MAX(retweet_count) as max_retweets FROM twitter GROUP BY id) WHERE max_retweets > 100")
print('Number of tweets retweeted more than 100 times: %d' % count_retweeted_sql.first()[0])


Number of tweets retweeted more than 100 times: 15706


#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [22]:
print(cols[6])

location


We have to group the information by `id` and `location`, and once we have this grouping, we can consider the aggregation corresponding to his key taking the max number of retweets, and sort the data to get the top 3 per grouping.

##### Using RDD

In [25]:
tweets = rdd_twitter.map(lambda x: ((x[3], x[6]), x[8]))
tweets.take(2)

[(('1252251164227362821', 'India'), 5477),
 (('1252251164227362821', 'India'), 5477)]

In [29]:
#from operator import max
unique_tweets_country = tweets.reduceByKey(lambda a, b : a if a>b else b)
unique_tweets_country.take(2)

[(('1252251164227362821', 'India'), 5477),
 (('1252251164256555009', 'UK'), 6513)]

In [30]:
unique_tweets_ = unique_tweets_country.map(lambda x: (x[0][1], (x[0][0], x[1])))
unique_tweets_.take(2)


[('India', ('1252251164227362821', 5477)),
 ('UK', ('1252251164256555009', 6513))]

In [31]:
grouped_tweets = unique_tweets_.groupByKey()
grouped_tweets.take(2)

[('India', <pyspark.resultiterable.ResultIterable at 0x7ff4d20f5f30>),
 ('China', <pyspark.resultiterable.ResultIterable at 0x7ff4d20f5e10>)]

In [37]:
####
unique_tweets_rdd = unique_tweets_country.map(lambda x: (x[0][1], (x[1],x[0][0])))
unique_tweets_rdd.take(2)


[('India', (5477, '1252251164227362821')),
 ('UK', (6513, '1252251164256555009'))]

In [38]:
grouped_tweets_rdd = unique_tweets_rdd.groupByKey()
grouped_tweets_rdd.take(2)

[('India', <pyspark.resultiterable.ResultIterable at 0x7ff4d3b87310>),
 ('China', <pyspark.resultiterable.ResultIterable at 0x7ff4d3b86680>)]

In [40]:
from heapq import nlargest
n = 3
top_per_country_rdd = grouped_tweets_rdd.map(lambda x: (x[0], nlargest(n, x[1])))
print( top_per_country_rdd.collect() )

[('India', [(9988, '1252332114948874240'), (9976, '1252252336921206787'), (9973, '1252254519116746754')]), ('China', [(9998, '1252335780707684352'), (9993, '1252253596516843520'), (9984, '1252255562525560832')]), ('Mexico', [(9998, '1252253843145912320'), (9994, '1252255209776189442'), (9971, '1252252016006422533')]), ('USA', [(9994, '1252331777806524416'), (9987, '1252254239805579264'), (9982, '1252335464750735362')]), ('Pakistan', [(9988, '1252334264248606720'), (9975, '1252251912084357121'), (9973, '1252252126694309888')]), ('UK', [(9991, '1252333018578145280'), (9989, '1252252091822870529'), (9985, '1252254043973603329')]), ('Spain', [(9992, '1252335445876367361'), (9981, '1252334839094599681'), (9969, '1252254696112300032')]), ('Canada', [(9997, '1252335430323888128'), (9992, '1252254877939531776'), (9987, '1252252082825986051')]), ('Germany', [(9999, '1252334028092399622'), (9997, '1252330902325248000'), (9990, '1252252295510855682')]), ('Chile', [(9988, '1252253612140490759'), (

In [41]:
for country, tweet_ in top_per_country_rdd.collect():
    print("Country: {0}, Top Tweet IDs: {1}".format(country,tweet_))


Country: India, Top Tweet IDs: [(9988, '1252332114948874240'), (9976, '1252252336921206787'), (9973, '1252254519116746754')]
Country: China, Top Tweet IDs: [(9998, '1252335780707684352'), (9993, '1252253596516843520'), (9984, '1252255562525560832')]
Country: Mexico, Top Tweet IDs: [(9998, '1252253843145912320'), (9994, '1252255209776189442'), (9971, '1252252016006422533')]
Country: USA, Top Tweet IDs: [(9994, '1252331777806524416'), (9987, '1252254239805579264'), (9982, '1252335464750735362')]
Country: Pakistan, Top Tweet IDs: [(9988, '1252334264248606720'), (9975, '1252251912084357121'), (9973, '1252252126694309888')]
Country: UK, Top Tweet IDs: [(9991, '1252333018578145280'), (9989, '1252252091822870529'), (9985, '1252254043973603329')]
Country: Spain, Top Tweet IDs: [(9992, '1252335445876367361'), (9981, '1252334839094599681'), (9969, '1252254696112300032')]
Country: Canada, Top Tweet IDs: [(9997, '1252335430323888128'), (9992, '1252254877939531776'), (9987, '1252252082825986051')]


##### Using DataFrame

In [43]:
from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window

tweets_country_df = df_twitter2.groupBy(["location", "id"]).agg({"retweet_count": "max"}) \
                    .withColumnRenamed("max(retweet_count)", "max_retweets")
tweets_country_df.show()

+--------+-------------------+------------+
|location|                 id|max_retweets|
+--------+-------------------+------------+
|   China|1252251437469499393|         326|
|     USA|1252251573046173697|        3424|
|     USA|1252251719381237770|        7280|
|Pakistan|1252251741350961153|         857|
|   Chile|1252251780668325888|        4686|
|Pakistan|1252252009891106816|        8128|
|  Canada|1252252183787012098|        5912|
|Pakistan|1252252354813935616|        1968|
|  Mexico|1252252483516334080|        4640|
| Germany|1252252700936417280|        8919|
|Pakistan|1252252742346616834|        4616|
|  Canada|1252252862073262080|        6700|
|   Italy|1252252869379702788|        8316|
|  Mexico|1252252931371433985|        5645|
|  Mexico|1252252944977801217|        9269|
|  Mexico|1252252988468420608|        2648|
|   Spain|1252253024170450949|        4790|
|     USA|1252253058970652672|        8746|
|   Chile|1252253183604219904|        7569|
|   India|1252253380153683969|  

In [45]:
# Use a window function to rank the tweets by the max retweet count within each country
window = Window.partitionBy("location").orderBy(desc("max_retweets"))
ranked_tweets_df = tweets_country_df.withColumn("rank", rank().over(window))
ranked_tweets_df.show()

+--------+-------------------+------------+----+
|location|                 id|max_retweets|rank|
+--------+-------------------+------------+----+
|  Canada|1252335430323888128|        9997|   1|
|  Canada|1252254877939531776|        9992|   2|
|  Canada|1252252082825986051|        9987|   3|
|  Canada|1252336589809897473|        9977|   4|
|  Canada|1252251718617722881|        9969|   5|
|  Canada|1252331563351769094|        9966|   6|
|  Canada|1252335441950408704|        9959|   7|
|  Canada|1252254353764696064|        9947|   8|
|  Canada|1252251536346042369|        9945|   9|
|  Canada|1252330715066372102|        9929|  10|
|  Canada|1252331297302855686|        9928|  11|
|  Canada|1252251705783324672|        9926|  12|
|  Canada|1252252615108329473|        9925|  13|
|  Canada|1252251405668286466|        9920|  14|
|  Canada|1252254639434719239|        9915|  15|
|  Canada|1252252550327201792|        9908|  16|
|  Canada|1252255993184092161|        9893|  17|
|  Canada|1252331978

In [46]:
# Filter for the top 3 most retweeted tweets per country
top_tweets_country_df = ranked_tweets_df.filter("rank <= 3") \
    .select("location", "id", "max_retweets") \
    .orderBy("location", desc("max_retweets"))

# Show the results
top_tweets_country_df.show()


+--------+-------------------+------------+
|location|                 id|max_retweets|
+--------+-------------------+------------+
|  Canada|1252335430323888128|        9997|
|  Canada|1252254877939531776|        9992|
|  Canada|1252252082825986051|        9987|
|   Chile|1252253612140490759|        9988|
|   Chile|1252334891951427585|        9984|
|   Chile|1252253710182481920|        9978|
|   China|1252335780707684352|        9998|
|   China|1252253596516843520|        9993|
|   China|1252255562525560832|        9984|
| Germany|1252334028092399622|        9999|
| Germany|1252330902325248000|        9997|
| Germany|1252252295510855682|        9990|
|   India|1252332114948874240|        9988|
|   India|1252252336921206787|        9976|
|   India|1252254519116746754|        9973|
|   Italy|1252252106750377996|        9994|
|   Italy|1252251206027816960|        9984|
|   Italy|1252330500670337024|        9971|
|  Mexico|1252253843145912320|        9998|
|  Mexico|1252255209776189442|  

##### Using spark.sql and the temporay table.

In [47]:
# Get top 3 most retweeted tweets per country
top_tweets = spark.sql("SELECT location, id, retweet_count FROM \
                        (SELECT location, id, retweet_count, \
                         ROW_NUMBER() OVER (PARTITION BY location ORDER BY retweet_count DESC) as rank \
                         FROM twitter) \
                        WHERE rank <= 3")

# Show the result
top_tweets.show()


+--------+-------------------+-------------+
|location|                 id|retweet_count|
+--------+-------------------+-------------+
|   Chile|1252253612140490759|         9988|
|   Chile|1252253612140490759|         9988|
|   Chile|1252253612140490759|         9988|
|   China|1252335780707684352|         9998|
|   China|1252335780707684352|         9998|
|   China|1252335780707684352|         9998|
| Germany|1252334028092399622|         9999|
| Germany|1252334028092399622|         9999|
| Germany|1252334028092399622|         9999|
|   India|1252332114948874240|         9988|
|   India|1252332114948874240|         9988|
|   India|1252332114948874240|         9988|
|   Italy|1252252106750377996|         9994|
|   Italy|1252252106750377996|         9994|
|   Italy|1252252106750377996|         9994|
|  Canada|1252335430323888128|         9997|
|  Canada|1252335430323888128|         9997|
|  Canada|1252335430323888128|         9997|
|  Mexico|1252253843145912320|         9998|
|  Mexico|

#### 2.9 Total number of tweets per country

##### Using RDD (3 points)

We have to find the unique tweets per country, and then count them. 

In [48]:
from operator import add
tweets = rdd_twitter.map(lambda x: ((x[6], x[3]), 1))
unique_tweets_country = tweets.reduceByKey(add)
unique_tweets_country.take(2)

[(('India', '1252251164227362821'), 15894),
 (('UK', '1252251164256555009'), 15894)]

In [50]:

## Equivalent to
## total_tweets = tweets.reduceByKey(lambda a,b : a+b)
## checking if it is faster

tweets_per_country = unique_tweets_country.map(lambda x: (x[0][0], 1)) # (x[1], x[0][1], 1)))
total_tweets_country = tweets_per_country.reduceByKey(add)
total_tweets_country.take(2)

[('India', 1480), ('China', 1457)]

In [51]:

for country, tweet_count in total_tweets_country.collect():
    print("Country: {0}, Tweet Count: {1}".format(country,tweet_count))


Country: India, Tweet Count: 1480
Country: China, Tweet Count: 1457
Country: Mexico, Tweet Count: 1408
Country: USA, Tweet Count: 1538
Country: Pakistan, Tweet Count: 1470
Country: UK, Tweet Count: 1375
Country: Spain, Tweet Count: 1463
Country: Canada, Tweet Count: 1441
Country: Germany, Tweet Count: 1426
Country: Chile, Tweet Count: 1409
Country: Italy, Tweet Count: 1422


##### Using DataFrame (2 points)

In [53]:
# Filter out duplicate tweets per location
unique_tweets_per_country = df_twitter2.dropDuplicates(["location", "id"])

# Count the number of unique tweets per country
tweets_per_country = unique_tweets_per_country.groupBy("location") \
    .agg({"id": "count"}) \
    .withColumnRenamed("count(id)", "tweet_count")

# Show the results
tweets_per_country.show()


+--------+-----------+
|location|tweet_count|
+--------+-----------+
| Germany|       1426|
|   China|       1457|
|   India|       1480|
|   Chile|       1409|
|   Italy|       1422|
|   Spain|       1463|
|     USA|       1538|
|  Mexico|       1408|
|      UK|       1375|
|  Canada|       1441|
|Pakistan|       1470|
+--------+-----------+



##### Using spark.sql and the temporay table. (1 point)

In [54]:
# Count the number of distinct tweets per country using Spark SQL
distinct_tweets_per_country = spark.sql("SELECT DISTINCT location, id \
                                         FROM twitter")

tweets_per_country = distinct_tweets_per_country.groupBy("location") \
                                                .count() \
                                                .withColumnRenamed("country", "tweet_count")

# Show the result
tweets_per_country.show()


+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   China| 1457|
|   India| 1480|
|   Chile| 1409|
|   Italy| 1422|
|   Spain| 1463|
|     USA| 1538|
|  Mexico| 1408|
|      UK| 1375|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [17]:
df_twitter2.dtypes

[('created_at', 'string'),
 ('favorite_count', 'bigint'),
 ('hashtags', 'array<string>'),
 ('id', 'string'),
 ('in_reply_to_status_id', 'string'),
 ('in_reply_to_user_id_str', 'string'),
 ('location', 'string'),
 ('reply_count', 'bigint'),
 ('retweet_count', 'bigint'),
 ('source', 'string'),
 ('user_id_str', 'string'),
 ('user_followers_count', 'bigint'),
 ('user_friends_count', 'bigint'),
 ('user_created_at', 'string'),
 ('extracted_source', 'string')]

Observe that the column `hashtags` is of type `array<string>` which cannot be converted correctly to the dataframe we want to store in the csv. Therefore, we may need to convert it to a string frist

In [18]:
from pyspark.sql.functions import concat_ws

# Replace 'array_column' with the actual name of your array column
df_twitter3 = df_twitter2.withColumn('string_hashtags', concat_ws(',', 'hashtags'))

df_twitter3.dtypes

[('created_at', 'string'),
 ('favorite_count', 'bigint'),
 ('hashtags', 'array<string>'),
 ('id', 'string'),
 ('in_reply_to_status_id', 'string'),
 ('in_reply_to_user_id_str', 'string'),
 ('location', 'string'),
 ('reply_count', 'bigint'),
 ('retweet_count', 'bigint'),
 ('source', 'string'),
 ('user_id_str', 'string'),
 ('user_followers_count', 'bigint'),
 ('user_friends_count', 'bigint'),
 ('user_created_at', 'string'),
 ('extracted_source', 'string'),
 ('string_hashtags', 'string')]

In [20]:
df_twitter4 = df_twitter3.drop('hashtags')
df_twitter4.dtypes

[('created_at', 'string'),
 ('favorite_count', 'bigint'),
 ('id', 'string'),
 ('in_reply_to_status_id', 'string'),
 ('in_reply_to_user_id_str', 'string'),
 ('location', 'string'),
 ('reply_count', 'bigint'),
 ('retweet_count', 'bigint'),
 ('source', 'string'),
 ('user_id_str', 'string'),
 ('user_followers_count', 'bigint'),
 ('user_friends_count', 'bigint'),
 ('user_created_at', 'string'),
 ('extracted_source', 'string'),
 ('string_hashtags', 'string')]

In [ ]:
# Using DataFrame
# Write to CSV partitioned by 'location'
df_twitter4.coalesce(1).write.partitionBy('location').format('csv').option('header', 'true').mode('overwrite').save('/stackOut/')

#df_twitter2.write.partitionBy("location").csv("tweets_per_country")


This code keeps giving me an error that I think is related with the memory space. But in case of enough space this code should perform correctly.

#### 3.2 Save the data as parquet files (1 points)

In [ ]:
# Using DataFrame

df_twitter2.write.partitionBy("location").mode("overwrite").parquet("tweets_by_country.parquet")
